In [8]:
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, ContinuousColorMapper, LinearColorMapper, ColorBar, LabelSet, Label
from bokeh.palettes import Spectral10
from bokeh.plotting import figure, show, output_notebook
from bs4 import BeautifulSoup
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, normalize
from transformers import AutoTokenizer
import gensim.downloader
import ipywidgets as widgets
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import requests
import seaborn as sns
import string
import sys

In [9]:
from nltk.corpus import stopwords
import spacy

stopwoorden = set(stopwords.words('dutch'))

def normalize_numbers(tekst):
    """
    Normaliseert getallen in de tekst, zoals telefoonnummers, postcodes en huisnummers.
    """
    tekst = re.sub(r'(\+31|0)[-\s]*(6-?[\d\s-]{8}|7[\d\s-]{8}|1[\d\s-]{8})', '***', tekst)
    tekst = re.sub(r'\b\d{4}\s*[a-zA-Z]{2}\b', '***', tekst)
    tekst = re.sub(r'\b\d+(?:\s*[a-zA-Z])?\b', '***', tekst)
    return tekst

def converteer_naar_kleine_letters(tekst):
    """
    Converteert de tekst naar kleine letters.
    """
    return tekst.lower()

def verwijder_niet_alfanumeriek(tekst):
    """
    Verwijdert niet-alfanumerieke tekens uit de tekst.
    """
    gefilterde_tekst = re.sub(r'[^\w\s]', '', tekst)
    return gefilterde_tekst

def verwijder_witruimte(tekst):
    """
    Verwijdert witruimte aan het begin en einde van de tekst.
    """
    return tekst.strip()

def verwijder_stopwoorden(woorden):
    """
    Verwijdert stopwoorden uit de lijst van woorden.
    """
    gefilterde_woorden = [woord for woord in woorden if woord.lower() not in stopwoorden]
    return gefilterde_woorden

def tokenize_zin_en_woorden(tekst):
    """
    Splitst de tekst op in zinnen en woorden.
    """
    zinnen = sent_tokenize(tekst)
    woorden = word_tokenize(tekst)
    return zinnen, woorden

def lemmatize_tekst(tekst):
    """
    Lemmatiseert de tekst door de woorden terug te brengen naar hun basisvorm.
    """
    doc = nlp(tekst)
    lemmatized_tokens = [token.lemma_ for token in doc]
    lemmatized_text = " ".join(lemmatized_tokens)
    return lemmatized_text

## Keyword filtering op fictief interview zorgspecialisten

Om te beginnen testen we op test document. We hebben de volgende vraag aan Chat-GPT gesteld om aan dit document te komen:

> Kun je voor mij een interview transcriptie genereren met daarin de volgende onderwerpen:
> - medicijnen
> - boodschappen
> - dementie
> - huisarts
> - mantelzorger

### Hypothese
Alle onderwerpen die aan Chat-GPT zijn gevraagd om in de tekst te verwerken, komen terug in de uitkomst van de keyword extrapolatie.

### Aanpak testen van hypothese
We defineren een methode welke 2 parameters heeft, namelijk:
- `text`, een lijst van strings waarop de keyword extrapolatie methode op wordt toegepast
- `keywords_per_phrase`, een heel getal waarop gebaseerd word hoeveel keywords er per item uit de gegeven `text` gehaald moeten worden

De uitkomst van deze functie is een lijst met steek worden uit de gegeven `text`.

We vergelijken de uitkomst met de lijst van onderwerpen gevraagd aan Chat-GPT om deze te verwerken in het test document.

In [10]:
test_doc = """
Interviewer: Goedemiddag, vandaag praten we over een aantal belangrijke onderwerpen met betrekking tot gezondheidszorg en de rol van mantelzorgers. Laten we beginnen met medicijnen. Welke rol spelen medicijnen in de behandeling van bepaalde aandoeningen?

Expert: Bedankt voor de vraag. Medicijnen spelen een cruciale rol bij de behandeling van diverse aandoeningen. Ze kunnen worden gebruikt om symptomen te verlichten, ziekten te beheersen en zelfs levens te redden. Geneesmiddelen worden voorgeschreven door artsen en apothekers op basis van de diagnose van de patiënt.

Interviewer: Begrijpelijk. Nu willen we graag het onderwerp boodschappen aansnijden. Hoe kunnen mantelzorgers helpen bij het doen van boodschappen voor hun zorgbehoevende familielid?

Expert: Mantelzorgers kunnen een belangrijke rol spelen bij het doen van boodschappen voor hun zorgbehoevende familielid. Ze kunnen helpen bij het opstellen van een boodschappenlijstje, winkelen voor de benodigde items en zelfs zorgen voor het bezorgen van de boodschappen aan huis. Dit verlicht de last voor de persoon die verzorgd wordt en zorgt ervoor dat essentiële benodigdheden gemakkelijk beschikbaar zijn.

Interviewer: Interessant. Laten we nu overgaan naar het onderwerp dementie. Hoe kan een mantelzorger omgaan met de uitdagingen die dementie met zich meebrengt?

Expert: Dementie kan inderdaad uitdagend zijn voor zowel de persoon met dementie als voor de mantelzorger. Het is belangrijk voor mantelzorgers om geduldig en begripvol te zijn. Ze kunnen leren over dementie, ondersteuning zoeken bij professionele zorgverleners en deelnemen aan ondersteuningsgroepen. Het bieden van een gestructureerde en veilige omgeving, het stimuleren van cognitieve activiteiten en het handhaven van een goede communicatie kunnen ook helpen bij het omgaan met de uitdagingen van dementie.

Interviewer: Bedankt voor de waardevolle informatie. Tot slot, laten we het hebben over de rol van de huisarts. Wat is de rol van de huisarts bij de zorg voor mensen met langdurige gezondheidsproblemen?

Expert: De huisarts speelt een essentiële rol bij de zorg voor mensen met langdurige gezondheidsproblemen. Ze zijn vaak het eerste aanspreekpunt voor de patiënt en coördineren de zorg voor de patiënt. De huisarts kan de gezondheidstoestand beoordelen, diagnoses stellen, behandelingen voorschrijven en doorverwijzen naar specialisten indien nodig. Ze kunnen ook een vertrouwensband opbouwen met de patiënt en de mantelzorger, en hen ondersteunen bij het omgaan met de uitdagingen van de zorg.

Interviewer: Heel erg bedankt voor uw tijd en expertise. We hebben waardevolle inzichten gekregen over medicijnen, boodschappen, dementie, de rol van de huisarts en mantelzorgers.

Expert: Graag gedaan. Het was een genoegen om hierover te praten. Ik hoop dat deze informatie nuttig zal zijn voor degenen die betrokken zijn bij de zorg voor hun dierbaren.
"""

In [11]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import string

nlp = spacy.load("nl_core_news_lg")
punctuations = string.punctuation

def keyword_filter_tokenize(text):
    tokens = nlp(str(text))
    keywords = []

    for token in tokens:
        if token.is_punct or token.is_currency:
            continue
        elif token.pos_ in ["NOUN", "PROPN", "ADJ"]:
            keywords.append(token.lemma_)
    return keywords


def keyword_filter_preprocessing(text):
    if isinstance(text, tuple):
        text = ' '.join(str(t) for t in text)

    text = converteer_naar_kleine_letters(text)
    text = normalize_numbers(text)
    text = verwijder_niet_alfanumeriek(text)
    text = verwijder_witruimte(text)
    text = lemmatize_tekst(text)
    return text

In [12]:
def get_keywords(text, keywords_per_phrase = 1):
    keyword_vectorizer = TfidfVectorizer(tokenizer=keyword_filter_tokenize, preprocessor=keyword_filter_preprocessing)
    keyword_matrix = keyword_vectorizer.fit_transform(text)
    keyword_feature_names = keyword_vectorizer.get_feature_names_out()

    useful_keywords = []
    for i, document in enumerate(keyword_matrix):
        tfidf_scores = keyword_matrix[i].toarray()[0]
        top_indices = tfidf_scores.argsort()[-keywords_per_phrase:][::-1]
        top_keywords = [keyword_feature_names[idx] for idx in top_indices]
        useful_keywords.extend(top_keywords)

    useful_keywords = list(set(useful_keywords))  # Remove duplicate keywords
    return useful_keywords

In [13]:
get_keywords(test_doc.split("\n"), keywords_per_phrase = 1)

['nuttig',
 'huisarts',
 'patiënt',
 'dementie',
 'zorgbehoevend',
 'erg',
 'medicijn',
 'boodschap',
 'cruciaal',
 'zorgverlener']